In [1]:
import datetime as dt
import pandas as pd
import pymongo
import requests
import string

from config import API_KEY
from countries import countries, country_codes

from rake_nltk import Rake, Metric
from py_translator import Translator


In [2]:
#connect to mongodb
conn = 'mongodb://localhost:27017/top_headlines'
client = pymongo.MongoClient(conn)
# Declare the database
db = client["top_headlines"]
# Declare the collection
collection = db["countries_data"]

In [3]:
#get data
data = collection.find()

#get alphabet list
alphabet = list(string.ascii_lowercase)

In [4]:
#initialize countryWords dict and allWords string
countryWords = {}
allWords = ''

In [8]:
# ### reset data
collection = db["countries_data"]

for i in range(len(country_codes)):
    print(countries[i])
    badList = data[i]["articles"]
    goodList = []
    for j in range(60):
        goodList.append(badList[j])
    print(len(goodList))
#     print(collection)
    print(collection.find({"country": countries[i]}))
    collection.update_one({"country": countries[i]}, {"$set": {"articles": goodList}})
    print(len(data[i]["articles"]))

Argentina
60
60
Australia
60
60
Austria
60
60
Belgium
60
60
Brazil
60
60
Bulgaria
60
60
Canada
60
60
China
60
60
Colombia
60
60
Cuba
60
60
Czech Republic
60
60
Egypt
60
60
France
60
60
Germany
60
60
Greece
60
60
Hong Kong
60
60
Hungary
60
60
India
60
60
Indonesia
60
60
Ireland
60
60
Israel
60
60
Italy
60
60
Japan
60
60
Latvia
60
60
Lithuania
60
60
Malaysia
60
60
Mexico
60
60
Morocco
60
60
Netherlands
60
60
New Zealand
60
60
Nigeria
60
60
Norway
60
60
Philippines
60
60
Poland
60
60
Portugal
60
60
Romania
60
60
Russia
60
60
Saudi Arabia
60
60
Serbia
60
60
Singapore
60
60
Slovakia
60
60
Slovenia
60
60
South Africa
60
60
South Korea
60
60
Sweden
60
60
Switzerland
60
60
Taiwan
60
60
Thailand
60
60
Turkey
60
60
United Arab Emirates
60
60
Ukraine
60
60
United Kingdom
60
60
United States of America
60
60
Venezuela
60
60


In [9]:
untranslated_countries = []
#for length of country_code list
for i in range(len(country_codes)):
    #visit news api
    query_url = f"https://newsapi.org/v2/top-headlines?country={country_codes[i]}&apiKey={API_KEY}"
    print("-----------------------------------------------------------")
    print(countries[i])
    print(query_url)
    #get response
    response = requests.get(query_url).json()

    #initialize countryWords dict and date
    countryWords[countries[i]] = ''
    date_list = []
    print("TITLES::::::")
    #for each article
    for article in response["articles"]:
        #set date and title
        date = article["publishedAt"][:10]
        date_list.append(date)
        title = article["title"].split(" - ")[0]
        print(title)
        #construct article dictionary
        articleDict = {"source": article["source"]["name"], 
                    "title": title, 
                    "url": article["url"], 
                    "date": date}
        #set the collection
        collection = db["countries_data"]
        #add article dict to countries data db
        collection.update_one({"country": countries[i]}, {"$push": {"articles": articleDict}})
        print("countries_data updated with " + countries[i] + " data.")
        #add title text to countryWords dict
        countryWords[countries[i]] = countryWords[countries[i]] + title + ' '
        ####end of articles loop
        ########################

    #untranslated string of article titles
    untranslated = countryWords[countries[i]]
#     print(untranslated)

#     try to translate
    try:
        translated = Translator().translate(text=untranslated, dest='en').text
        print("TRANSLATED::::::")
        print(countries[i])
        print(translated)

        #set the collection
        collection = db["countries_words"]
        #add translated string to countries_words collection
        collection.update_one({"country": countries[i]}, {"$set": {max(date_list): translated}}, upsert=True)
        print("countries_words updated with " + countries[i] + " words for " + max(date_list))
    except:
        untranslated_countries.append(countries[i])
        print(countries[i] + " failed to translate")

    #add translated string to string of all titles
    allWords = allWords + translated

    #set rake settings
    r = Rake(max_length=2, ranking_metric=Metric.WORD_DEGREE)
    #extract keywords with scores from translated string
    r.extract_keywords_from_text(translated)
    phrasesScores = r.get_ranked_phrases_with_scores()
    # print(phrasesScores)

    #initialize keywords dict
    keywords_dict = {"keywords": {}}
    #for index and keyword-score pair,
    for j, pair in enumerate(phrasesScores):

        #format pair into score and word
        pair = list(pair)
        score = pair[0]
        word = pair[1]

        #set special character list
        special_chars = ["-", "'", ":", ".", "’", "‘", "\\", "(", ")", "”", "“", "!", "?", "【", "】", "\""]
        # print(j, word)
        #if any special characters in word or word less than 2 characters,
        if any(x in word for x in special_chars) or len(word) < 2:
            # print(j)
            #remove word-score pair
            phrasesScores.pop(j)

        else:
            #for character in word
            for char in word:
                #if any letter matches character,
                if any(letter == char for letter in alphabet):
                    #word has letters, exit loop
                    hasLetters = True
                    break
                else:
                    hasLetters = False

            #if word has letters, 
            if hasLetters == True:
                #add word-score pair to keywords_dict
                keywords_dict["keywords"][word] = score

            #if word has no letters,
            else:
                #remove word-score pair
                # print(word + " has no letters")
                phrasesScores.pop(j)

    #set collection
    collection = db["countries_keywords"]
    #update countries_keywords collection with keywords_dict
    collection.update_one({"country": countries[i]}, {"$set": {"keywords": keywords_dict["keywords"]}}, upsert=True)
    ###end of country loop
    ######################

-----------------------------------------------------------
Argentina
https://newsapi.org/v2/top-headlines?country=ar&apiKey=89f29e52b3ff48fbbdb6b5439cf0fba4
TITLES::::::
Macri invitó a un K y a una diputada massista al viaje a la India, pero le dijeron que no
countries_data updated with Argentina data.
Marcelo D’Alessio “es el engranaje de algo más importante” | La abogada de Echebest habló de la denuncia...
countries_data updated with Argentina data.
Emiliano Sala: comenzó en Progreso el velorio del ex futbolista argentino del Nantes y el Cardiff
countries_data updated with Argentina data.
Francia vive un nuevo fin de semana de protestas de los chalecos amarillos
countries_data updated with Argentina data.
El descargo de Calu Rivero después de la escena de violación en "Campanas en la noche"
countries_data updated with Argentina data.
San Lorenzo pretende ganar por primera vez con Jorge Almirón ante Newell's: hora, TV y formaciones
countries_data updated with Argentina data.
Murió Br

TITLES::::::
Verhärtete Fronten in München: Merkel rechnet mit Trump ab
countries_data updated with Austria data.
Jet mit „Friends“-Stars Aniston und Cox musste notlanden
countries_data updated with Austria data.
Liechtenstein: Der Schatz des Fürsten
countries_data updated with Austria data.
Aufregung um Verleihung von Grazer Ehrenzeichen an Gabalier
countries_data updated with Austria data.
Schüsse in Wien-Meidling: Frau in Krankenhaus gestorben
countries_data updated with Austria data.
Aktuelle Studie weist die Vorteile von Vitamin-D auf den Blutzuckerspiegel nach
countries_data updated with Austria data.
Ski-WM live: Slalom der Damen in Aare
countries_data updated with Austria data.
Ski-WM: Liensberger lauert auf Slalommedaille
countries_data updated with Austria data.
Hirscher nach Silber: "Ich kann nicht immer einspringen
countries_data updated with Austria data.
Papst entlässt früheren Erzbischof, weiterer Missbrauchs-Verdachtsfall
countries_data updated with Austria data.
Schaus

TRANSLATED::::::
Brazil
Onyx meets with Bolsonaro in the Alvorada Vasco and Fluminense meet with Ferj to decide on South sector of the Maracanã for the end Yellow vests return to the streets of Paris in the 14th weekend of protests Body of boy who died after being immobilized in supermarket of Rio by The city of Ceará, Brazil, is one of the most important cities in the country, and it is known as the "untouchable" state of Ceará. millimeters President of Petrobras wants to sell almost everything in the refining American ceases to be cardinal and priest for sexual abuse 'The disloyal will live waiting for the world to fall into his head', says message published by Bebianno The automaker that continue selling cars and not mileage will break, says founder of Waze Carille plans to recover Clayson and have speed in Timão Former mayor dies with suspicion of dengue he Rizia is embarrassed during sex talk at dawn Do you have a weekend flight? Stay tuned for the end of summer time Election in N

TITLES::::::
「傳統文化是珍寶」 公司老闆讚神韻非凡
countries_data updated with China data.
先南下再西游，蚂蚁金服的海外投资如何开花？
countries_data updated with China data.
铁树终开花 迪巴拉与C罗首次携手取得进球
countries_data updated with China data.
朝鲜举办活动纪念金正日诞辰77周年-中新网
countries_data updated with China data.
福州一自建民房意外倒塌：15人获救送医民众回忆如地震一般-中新网视频
countries_data updated with China data.
西班牙人客战瓦伦西亚大名单：武磊领衔有望首发
countries_data updated with China data.
司机不赚钱滴滴喊巨亏，不差钱的滴滴到底把钱烧哪了？
countries_data updated with China data.
特朗普：美中谈判非常顺利 暗示或延长谈判大限
countries_data updated with China data.
【国际锐评】习近平首见美方代表两国元首引领经贸“加减法”
countries_data updated with China data.
国际观察：慕安会凸显跨大西洋矛盾-新华网
countries_data updated with China data.
习近平会见美国贸易代表莱特希泽和财政部长姆努钦
countries_data updated with China data.
全明星新秀赛美国队击败世界队-新华网体育
countries_data updated with China data.
“中国冬奥冠军之乡”七台河的“冠军馆”16日开馆-中新网
countries_data updated with China data.
【央视快评】坚定不移走中国特色社会主义法治道路_新闻频道
countries_data updated with China data.
2月16日江苏省外三元生猪市场行情动态- 价为,外三元,上涨了,跌了,KG
countries_data updated with China data.
中国人民

TRANSLATED::::::
Cuba
Cubans who complete mission abroad vote for the new Constitution (+ photos) UN follows closely the situation in Haiti Maduro qualifies for successful military exercises in Venezuela NASA prepares the first flight of an electric-powered aircraft Japanese drums boom at Feria del Book of Havana A former dictator, a suspected magnate and the threat of Boko Haram: the great African power elects president Oil prices rise in the weekly closing of markets Cuba beats 11x6 stars of the Salvadoran baseball league Taekwondoca Glenhis Hernández is ready for face the multi-sporting event Lima-2019 Putting the ball in its place Nicolás Maduro called on the military to prepare a special deployment plan in the face of the threats of Trump and Du Darán on Saturday in Cienfuegos to recognize the awards of the Latin American Audio Monologue Festival: " We are dreaming, but there are things that can be done "Nubia Alpha: mobile technology of the future (+ Video) The trade surplus of t

...

TITLES::::::
7 reasons you should buy a Nintendo Switch instead of a PlayStation 4 or an Xbox One in 2019
countries_data updated with South Africa data.
Zimbabwean rescuers pull out eight trapped miners alive
countries_data updated with South Africa data.
Small businesses hard hit by load shedding
countries_data updated with South Africa data.
Fearless Sri Lanka stun Proteas in Kingsmead epic
countries_data updated with South Africa data.
#MWC2019:The LG G8 ThinQ uses a vibrating OLED screen as a speaker
countries_data updated with South Africa data.
ISIS teen's wish to return stirs UK debate over jihadi brides
countries_data updated with South Africa data.
‘Virgil Vries did not grab his chance‚’ says Kaizer Chiefs coach Middendorp
countries_data updated with South Africa data.
Two killed, 71 injured after bus and car collide
countries_data updated with South Africa data.
'You can postpone an election, but you cannot postpone destiny'
countries_data updated with South Africa data.
ANC:

TITLES::::::
Mikaela Shiffrin ist Slalom-Weltmeisterin | NZZ
countries_data updated with Switzerland data.
Herzogin Meghan reist hochschwanger nach Marokko
countries_data updated with Switzerland data.
Betrunkener (26) landet in Betonsockel einer Bahnschranke
countries_data updated with Switzerland data.
Aktuelle Studie weist die Vorteile von Vitamin-D auf den Blutzuckerspiegel nach
countries_data updated with Switzerland data.
Sicherheitskonferenz: Merkel rechnet in München mit Trump ab
countries_data updated with Switzerland data.
Bruno Ganz (†77) ist tot
countries_data updated with Switzerland data.
Wie der Gigant Decathlon den Schweizer Sportmarkt aufmischen will | NZZ
countries_data updated with Switzerland data.
Kritik nach Notstand-Entscheid
countries_data updated with Switzerland data.
10 Gebote für WhatsApp-User
countries_data updated with Switzerland data.
Trumps Wahlkampfmanager soll lange in Haft
countries_data updated with Switzerland data.
Schiesserei auf Fabrikgelände
co

TRANSLATED::::::
Thailand
Negotiating "China-US" is not a chance to win the "Color-Trump" leader. Derby failed to defeat Brighton 1-2, the FA Cup, the U22 War, the practice before the Timor Duel. The first game was not an easy task. "Kanok" beat the strength of "Thanathorn", tackling the new generation. Don't be fascinated by the current Gold TV 2561, the drama, outstanding drama, O Chao, smooth! Bella is delighted to bring the female sergeant to the hero! Army Rua Saeng Agriculture 2-1, Nong Bua Chae Chabra Gold 2019: Training for amphibious fighting tactics, 1st prize, Rayong 18 l. 2 young men, Heng Mia tells another number, dating to "Songkran" without name Emerged from the Thai party, preserving the nation after Big Fort to resign from the police Survive 1 life, because first down! 'Pick-up' Setk Khlong-Jensen 5 Funeral Coaches Take the Main Asian Cup, Go Four Sides in China - Hope to Experience the Giant Duel Prognosis Weekly 17-23 Feb Watch out for "whooping cough" outbreaks. Add

TITLES::::::
Клімкін сказав, коли Росії чекати персональних санкцій від ЄС
countries_data updated with Ukraine data.
Уляна Супрун: почему не призначають міністром охорони здоров'я
countries_data updated with Ukraine data.
Астрономи зафіксували спалах зірки в мільярди разів яскравішої за Сонце
countries_data updated with Ukraine data.
Порошенко і Меркель обговорили протидію втручанню РФ у вибори
countries_data updated with Ukraine data.
Меркель виступила проти ізоляції Росії
countries_data updated with Ukraine data.
Пенсії зростуть: в уряді нагадали, коли і на скільки
countries_data updated with Ukraine data.
Промоутер Усика планує організувати бій українця з черговим росіянином | Бокс
countries_data updated with Ukraine data.
З явилися подробиці стрілянини в американській компанії
countries_data updated with Ukraine data.
У відповідь на ворожі обстріли бійці ООС знищили сімох окупантів
countries_data updated with Ukraine data.
NASA попрощалося з легендарним марсоходом Opportunity: оста

TRANSLATED::::::
United States of America
Man kills five in warehouse shooting spree shortly after being fired, Illinois police say Vatican Defrocks Former Cardinal McCarrick, Finds Him Guilty Of Sex Abuse Elon Musk says SpaceX is developing a 'bleeding' heavy-metal rocket ship. Making it work may be 100 times as hard as NASA's most difficult Mars mission, one expert says. Washington state House committee passes bill to ban personal, philosophical vaccine exemptions | TheHill Oscars reverses hugely unpopular decision after backlash from Brad Pitt, George Clooney and more The Morning After: Amazon drops 'LotR' hints Payless ShoeSource is closing all of its 2100 remaining U.S. stores Amazon’s HQ2 New York plans didn’t need to end this way From Echo to Ring doorbell and Fire TV, are you comfortable Amazon controlling your smart home? Ben Simmons Puts Up 28 Points & 6 Assists In Rising Stars Game | 2019 NBA All-Star Space Photos of the Week: The Trail of Opportunity and More In Munich, Pen

In [ ]:
# ####remove articles added to keywords collection
# collection = db["countries_keywords"]
# collection.update_many({}, {'$unset': {'articles':1}}, multi=True)

In [ ]:
# # ### reset data
# collection = db["country_data"]

# # for i in range(len(country_codes)):
# print("Argentina")
# badList = data[0]["articles"]
# goodList = []
# for i in range(40):
#     goodList.append(badList[i])
# print(len(goodList))
# collection.insert({"country": "AAAAAAAAAAA"}, {"$set": {"articles": goodList}})
# # collection.update_one({"country": countries[i]}, {"$set": {"keywords": keywords_dict["keywords"]}}, upsert=True)
# print(len(data[0]["articles"]))

In [13]:
response = requests.get(f"https://newsapi.org/v2/top-headlines?country=us&apiKey={API_KEY}").json()

In [14]:
response

{'status': 'error',
 'code': 'rateLimited',
 'message': 'You have made too many requests recently. Developer accounts are limited to 1,000 requests over a 24 hour period (250 requests available every 6 hours). Please upgrade to a paid plan if you need more requests.'}